# Targeted Cluster Analysis using cBLASTER

In [80]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import glob
from Bio import Phylo
from Bio import SeqIO
import math

### Define Project and folder layour

In [81]:
# Project Name
project="NCBI_P8A2_20230216"
BGC_group = 'Azoxy'
dir_applied_tools= project+"/00_applied_tools/"
dir_clinker = dir_applied_tools+"clinker/"
dir_cblaster = dir_applied_tools+"cblaster/"
dir_iTOL = dir_applied_tools+"iTOL/"

#### Make folders

In [82]:
directories = (dir_applied_tools, dir_clinker, dir_cblaster, dir_iTOL)
# Check whether the specified path exists or not
for path in directories:
   isExist = os.path.exists(path)
   if not isExist:
      # Create a new directory because it does not exist
      os.makedirs(path)
      print("Directory is created - "+path)

### Loading in cBLASTER and cLINKER results and creating overview table

Code below was automatically taking all cBLASTER hits, Ideally we need to have core genes identified that i have done for now manually, meaning genes present in all of the input clusters

In [83]:
# Code written together with thom to create the gene matrix automatically
df_genes_list = pd.read_csv(dir_clinker +'alignments.csv',header=None,usecols=[0,1]).dropna()
df_genes_list.rename( columns={0 :'Pair_gene_1'}, inplace=True )
df_genes_list.rename( columns={1 :'Pair_gene_2'}, inplace=True )
# Create an empty list
genes_list =[]
# Iterate over each row
for rows in df_genes_list.itertuples():
    # Create list for the current row
    each_pair =[rows.Pair_gene_1, rows.Pair_gene_2]
    # append the list to the final list
    genes_list.append(each_pair)

gene_group_list = [{genes_list[0][0]}]
for genes in genes_list:
    genes_added = False
    for gene_group in gene_group_list:
        if genes[0] in gene_group or genes[1] in gene_group:
            gene_group.add(genes[1])
            gene_group.add(genes[0])
            genes_added = True
    if genes_added == False:
        gene_group_list.extend([{genes[0]}])
        for gene_group in gene_group_list:
            if genes[0] in gene_group or genes[1] in gene_group:
                gene_group.add(genes[1])
                gene_group.add(genes[0])
sorted(gene_group_list)
gene_group_list.sort(key = len, reverse=True)

clean_gene_group_dictionary = {}
for index, group in enumerate(gene_group_list, start = 1):
    x = str(index).zfill(3) + "-" + str(len(group)).zfill(2)
    temp_group_dict = dict.fromkeys(group, x)
    clean_gene_group_dictionary.update(temp_group_dict)

gene_group_list

[{'EHGBBDDM_28800', 'PBHDNOKL_17950', 'PU648_09280', 'ady7', 'vlmA'},
 {'EHGBBDDM_28930', 'PBHDNOKL_17790', 'PU648_09235', 'ady3', 'vlmH'},
 {'EHGBBDDM_28900', 'PBHDNOKL_17820', 'PU648_09285', 'ady6', 'vlmO'},
 {'EHGBBDDM_28880', 'PBHDNOKL_17840', 'PU648_09275', 'ady8', 'vlmB'},
 {'EHGBBDDM_28890', 'PBHDNOKL_17830', 'PU648_09255', 'ady10', 'vlmR'},
 {'EHGBBDDM_28920', 'PBHDNOKL_17800', 'PU648_09245', 'ady4'},
 {'EHGBBDDM_28910', 'PBHDNOKL_17810', 'PU648_09250', 'ady5'},
 {'EHGBBDDM_28840', 'PBHDNOKL_17930', 'PU648_09270', 'ady9'},
 {'EHGBBDDM_28830', 'PBHDNOKL_17960', 'vlmI'},
 {'EHGBBDDM_28940', 'PBHDNOKL_17780', 'ady2'},
 {'EHGBBDDM_28970', 'PBHDNOKL_17730', 'orf+1'},
 {'EHGBBDDM_28960', 'PBHDNOKL_17700', 'PBHDNOKL_17710'},
 {'PBHDNOKL_17880', 'vlmC'},
 {'PBHDNOKL_17680', 'vlmJ'},
 {'PBHDNOKL_17660', 'vlmK'},
 {'PBHDNOKL_17890', 'vlmL'},
 {'PU648_09290', 'ady1'},
 {'PU648_09300', 'ady11'},
 {'PU648_09295', 'ady12'},
 {'EHGBBDDM_28980', 'PBHDNOKL_17750'},
 {'EHGBBDDM_28990', 'PBHDNOKL

In [84]:
# following code takes creates list of genes that share similartiy between the input clusters by analysing clinker alignment .csv file
#      note, that if two of the same clusters are uploaded, all genes in the BGC will be marked as shared genes and therefore will not be unique.
#           It could be considered to put score for each gene, eg. between how many clusters this gene is shared, thus indicating if it is essential or tailoring gene
df = pd.read_csv(dir_clinker +'alignments.csv',header=None,usecols=[0,1])
clinker_hit_gene_df = pd.concat([df.iloc[:,0],df.iloc[:,1]])
clinker_hit_gene_df =pd.DataFrame(clinker_hit_gene_df).set_axis(['clinker_hit_genes'], axis=1, inplace=False).drop_duplicates()
clinker_hit_gene_list = clinker_hit_gene_df['clinker_hit_genes'].values.tolist()
clinker_hit_gene_df.to_csv(dir_clinker+"clinker_hit_gene_df.csv",index=False)

#Core genes, table made manually
core_gene_df = pd.read_csv(dir_clinker +'manual_Core.csv',header=None)
core_gene_df =pd.DataFrame(core_gene_df).set_axis(['core_genes'], axis=1, inplace=False)
core_gene_list = core_gene_df['core_genes'].values.tolist()

C:\Users\matmal\AppData\Local\Temp\ipykernel_5784\2726100085.py:6: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  clinker_hit_gene_df =pd.DataFrame(clinker_hit_gene_df).set_axis(['clinker_hit_genes'], axis=1, inplace=False).drop_duplicates()
C:\Users\matmal\AppData\Local\Temp\ipykernel_5784\2726100085.py:12: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  core_gene_df =pd.DataFrame(core_gene_df).set_axis(['core_genes'], axis=1, inplace=False)


In [85]:
# cblaster processed files
cblaster_sample_overview = pd.read_csv(dir_cblaster+'jobs.txt', header=None)
cblaster_sample_overview.columns = ['Inputs']
cblaster_inputs_list = list(cblaster_sample_overview['Inputs'])
cblaster_inputs_list


# Code below can be used but needs to be activated below.
#Define minimum number of core genes identified by clinker for cluster to be kept
minimum_core_genes = 4
minimum_clinker_gene_hits = 5



In [86]:
# cblaster output file
cblaster_output = "data.csv"
# output tables
df_cblaster = pd.DataFrame(columns=['cblaster_input','gene_hits','clinker_gene_hits','%_clinker_gene_identity','%_average_gene_identity','Organism','Scaffold','Start','End','Score'])
for cblaster_input in cblaster_inputs_list:
    df=pd.read_csv(dir_cblaster+cblaster_input+"/"+cblaster_output)
    #df['%_cluster_gene_hits']=df.iloc[:, 5:].apply(np.count_nonzero, axis=1)/len(df.iloc[:, 5:].columns)
    df['gene_hits']=df.iloc[:, 5:].apply(np.count_nonzero, axis=1)
    df['%_average_gene_identity']=df.iloc[:, 5:-1].apply(np.nanmean, axis=1)
    df['cblaster_input'] = cblaster_input
    df_cblaster= pd.concat([df_cblaster, df])
# make subtable of cblaster outputs, filtering only gene hits by clinker
df = df_cblaster.copy()
df.drop(df.columns.difference(clinker_hit_gene_list), 1, inplace=True)


df['clinker_gene_hits'] = df[df > 0].count(axis=1)
df['%_average_gene_identity'] = df[df > 0].iloc[:,:-1].apply(np.nanmean, axis=1)
df_cblaster['clinker_gene_hits'] = df['clinker_gene_hits']
df_cblaster['%_clinker_gene_identity'] = df['%_average_gene_identity']
# sort the table and remove dublicated genome hits for different clusters, keeping only one with most hits
df_cblaster = df_cblaster.sort_values(by ='clinker_gene_hits', ascending = False)
df_cblaster = df_cblaster.drop_duplicates(subset=['Organism','Scaffold'], keep='first')
df_cblaster = df_cblaster[df_cblaster['clinker_gene_hits'] >= minimum_clinker_gene_hits]
df_cblaster = df_cblaster.sort_values(by =['cblaster_input','clinker_gene_hits'], ascending = False)
df_cblaster = df_cblaster.reset_index()

# Drop rows that do not contain core genes that are shered between all cblaster outputs
df = df_cblaster.copy()
df.drop(df.columns.difference(core_gene_list), 1, inplace=True)
df=df.replace(0, np.nan)
df.dropna(axis = 0, how = 'all', inplace = True)
df['core_gene_hits'] = df[df > 0].count(axis=1)
df = df[df['core_gene_hits'] >= minimum_core_genes]
df = pd.DataFrame(index=df.index)
df_cblaster = pd.merge(df, df_cblaster, left_index=True, right_index=True, how="inner")

# Save table
df_cblaster.to_csv(dir_cblaster+"df_cblaster.csv")
df_cblaster

C:\Users\matmal\AppData\Local\Temp\ipykernel_5784\2416689173.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(clinker_hit_gene_list), 1, inplace=True)
C:\Users\matmal\AppData\Roaming\Python\Python310\site-packages\pandas\core\apply.py:884: RuntimeWarning: Mean of empty slice
  results[i] = self.f(v)
C:\Users\matmal\AppData\Local\Temp\ipykernel_5784\2416689173.py:30: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(core_gene_list), 1, inplace=True)


,index,cblaster_input,gene_hits,clinker_gene_hits,%_clinker_gene_identity,%_average_gene_identity,Organism,Scaffold,Start,End,...,PU648_09325,PU648_09330,PU648_09335,PU648_09340,PU648_09345,PU648_09350,PU648_09355,PU648_09360,PU648_09365,PU648_09370
0,1832,P8-A2_Azodyrecin,36,12,99.908333,99.930556,GCF_007113545.2,NZ_CP041602.2,9821626,9898584,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,1825,P8-A2_Azodyrecin,36,12,99.908333,99.930556,GCF_007113485.2,NZ_CP041613.2,10165091,10242028,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,1835,P8-A2_Azodyrecin,36,12,99.675000,99.694444,GCF_007113565.2,NZ_CP041604.2,9832598,9910604,...,100.0,99.9,100.0,99.5,100.0,100.0,100.0,100.0,99.8,99.3
3,1837,P8-A2_Azodyrecin,36,12,99.583333,99.694444,GCF_007113585.2,NZ_CP041609.2,9576908,9652128,...,100.0,99.9,100.0,100.0,100.0,100.0,100.0,100.0,99.8,99.5
4,4591,P8-A2_Azodyrecin,34,12,98.141667,92.144444,GCF_900112965.1,NZ_FONR01000012,138273,176623,...,90.5,98.6,99.8,100.0,100.0,100.0,97.8,99.2,97.9,98.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,238,LC712332_Azodyrecin,8,8,51.425000,31.646154,GCF_014649335.1,NZ_BMSV01000002,727788,744654,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,82,LC712332_Azodyrecin,6,6,49.433333,22.815385,GCF_002214185.1,NZ_CP022161.1,5094814,5118039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,1007,AY116644_valanimycin,17,10,77.710000,66.500000,GCF_013414305.1,NZ_CP058693.1,118309,138736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,1579,AY116644_valanimycin,13,9,80.611111,52.815000,GCF_022698285.1,c00001_NZ_JALD..,9251988,9266505,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# iTOL visualization and splitting data into sub-tables
- To visalize the data, create a table where Each cluster is given its own color
    - Group the most similar clusters with XX cutoff to be responsible for production of best mach cBLASTER cluster
        - Probably best to use max cluster hits / with core gene hits to get estimated ratio.
            - This can create problems if input cluster is not part of the diamond database and better would be to use clinker identifited gene coutn. Requires to make table with cblaster_input and clinker hits
    - Low similarity clusters are offcolored to be something similar to target input
    - very-low similarity clusters are graphed as other related clusters.

## color gradient


Manually create label annotation names and rbg color codes that are linked to cblaster input names

Test with Thom
#Define cluster colors
df_labels = cblaster_sample_overview.copy()
df_labels['Label'] = ['BGC0001066','BGC0001819','rsn']
df_labels['Color code for rbg'] = ['0,158,115','230,159,0','0,114,178']
df_labels['HEX'] = ['#009e73','#e69f00','#0072b2']
df_labels['nr_of_core_genes'] = [10,7,10]
df_labels

In [88]:
#Define cluster colors
df_labels = cblaster_sample_overview.copy()
df_labels['Label'] = ['Valanimycin','Azodyrecin A-F','KA57-A','Eliomycin','Azodyrecin A-C']
df_labels['Color code for rbg'] = ['0,158,115','230,159,0','0,114,178','86,180,233','213,94,0']
df_labels['HEX'] = ['#009e73','#e69f00','#0072b2','#56b4e9','#d55e00']
df_labels['nr_of_core_genes'] = [10,13,25,19,12]
df_labels


,Inputs,Label,Color code for rbg,HEX,nr_of_core_genes
0,AY116644_valanimycin,Valanimycin,"0,158,115",#009e73,10
1,LC712332_Azodyrecin,Azodyrecin A-F,"230,159,0",#e69f00,13
2,NZ_AP018517.1_KA57-A,KA57-A,"0,114,178",#0072b2,25
3,NZ_CP027306.1_Elaiomycin,Eliomycin,"86,180,233",#56b4e9,19
4,P8-A2_Azodyrecin,Azodyrecin A-C,"213,94,0",#d55e00,12


Create iTOL annotation table

In [89]:
# Following code is used to split the data in the [df_cblaster] based on specific criteria

    # Criteria (currently not used)
bgc_product_core_genes_of_clinker = 0.95
bgc_prodyct_core_gene_identity = 0.9

bgc_cloesly_related_core_genes_of_clinker = 0.85
gc_cloesly_related_core_gene_identity = 0.7

bgc_other_genes_of_clinker = 0.1

    # Create three DataFrames

df_iTOL_cg = pd.DataFrame(columns=['Tree node (bgcflow input file name)','Color (rgba(255, 255, 255, 1.000))','Label'])
#Populate the table with df_cblaster results
#

df_iTOL_cg['Tree node (bgcflow input file name)'] = df_cblaster['Organism'] 
df_iTOL_cg['Label'] = df_cblaster['cblaster_input']
df_iTOL_cg['gene_hits'] = df_cblaster['gene_hits']
df_iTOL_cg['clinker_gene_hits'] = df_cblaster['clinker_gene_hits']
df_iTOL_cg['%_clinker_gene_identity'] = df_cblaster['%_clinker_gene_identity']
df_iTOL_cg['%_average_gene_identity'] = df_cblaster['%_average_gene_identity']

#Mrege in manually made Label translation table, with rbg codes, nr of core genes and label ids. Table index is changed to link the two tables
#
df_iTOL_cg = pd.merge(df_iTOL_cg.set_index('Label'),df_labels.set_index('Inputs'),left_index=True, right_index=True)
df_iTOL_cg['%_clinker_gene_hits'] = df_iTOL_cg['clinker_gene_hits'] / df_iTOL_cg['nr_of_core_genes']
#Transperency defines the number of core genes hit
df_iTOL_cg['transperency'] = df_iTOL_cg['%_clinker_gene_hits']*df_iTOL_cg['%_clinker_gene_identity']/100

#We want to color the hits based on the similarity to the original cluster aswell as transperancy product that is defined as nr of core hits X core gene identity, thus estimating similarity between two clusters. If transperency is below XX % then we wnat to set color to black. 
similarity_score = 0.5

df_iTOL_cg.loc[df_iTOL_cg['transperency'] <= similarity_score, 'Color code for rbg'] = '0, 0, 0'
df_iTOL_cg['Color (rgba(255, 255, 255, 1.000))'] = 'rgba('+df_iTOL_cg['Color code for rbg']+','+df_iTOL_cg['transperency'].apply(str)+')'


#Create the final table for iTOL export
#
df_iTOL_color_gradient = pd.DataFrame(columns=['Tree node','Color','Label'])
df_iTOL_color_gradient['Tree node'] = df_iTOL_cg['Tree node (bgcflow input file name)']
df_iTOL_color_gradient['Color'] = df_iTOL_cg['Color (rgba(255, 255, 255, 1.000))']
df_iTOL_color_gradient['Label'] = df_iTOL_cg['Label']
df_iTOL_color_gradient=df_iTOL_color_gradient.reset_index(drop=True)
df_iTOL_color_gradient.to_csv(dir_iTOL+"color_gradient.csv",index=False)


df_iTOL_cg

,Tree node (bgcflow input file name),"Color (rgba(255, 255, 255, 1.000))",gene_hits,clinker_gene_hits,%_clinker_gene_identity,%_average_gene_identity,Label,Color code for rbg,HEX,nr_of_core_genes,%_clinker_gene_hits,transperency
AY116644_valanimycin,GCF_013414305.1,"rgba(0,158,115,0.7771000000000001)",17,10,77.710000,66.500000,Valanimycin,"0,158,115",#009e73,10,1.000000,0.777100
AY116644_valanimycin,GCF_022698285.1,"rgba(0,158,115,0.7255000000000001)",13,9,80.611111,52.815000,Valanimycin,"0,158,115",#009e73,10,0.900000,0.725500
AY116644_valanimycin,GCF_014648775.1,"rgba(0,158,115,0.6312000000000001)",12,9,70.133333,42.410000,Valanimycin,"0,158,115",#009e73,10,0.900000,0.631200
LC712332_Azodyrecin,GCF_019933235.1,"rgba(230,159,0,0.9677692307692307)",13,13,96.776923,96.776923,Azodyrecin A-F,"230,159,0",#e69f00,13,1.000000,0.967769
LC712332_Azodyrecin,GCF_009176265.1,"rgba(230,159,0,0.9677692307692307)",13,13,96.776923,96.776923,Azodyrecin A-F,"230,159,0",#e69f00,13,1.000000,0.967769
...,...,...,...,...,...,...,...,...,...,...,...,...
P8-A2_Azodyrecin,GCF_026342395.1,"rgba(0, 0, 0,0.4227499999999999)",10,8,63.412500,18.027778,Azodyrecin A-C,"0, 0, 0",#d55e00,12,0.666667,0.422750
P8-A2_Azodyrecin,GCF_025398775.1,"rgba(0, 0, 0,0.4004166666666666)",11,8,60.062500,18.377778,Azodyrecin A-C,"0, 0, 0",#d55e00,12,0.666667,0.400417
P8-A2_Azodyrecin,GCF_027947595.1,"rgba(0, 0, 0,0.4174166666666666)",11,8,62.612500,18.883333,Azodyrecin A-C,"0, 0, 0",#d55e00,12,0.666667,0.417417
P8-A2_Azodyrecin,GCF_001514285.1,"rgba(0, 0, 0,0.2873333333333334)",8,7,49.257143,10.497222,Azodyrecin A-C,"0, 0, 0",#d55e00,12,0.583333,0.287333


## Shape plot
Use to visualize the gene architecture, eather by classifying which of the core shared genes belong to what function, thereby we can map percentage of core available

note: To make shape plot, make DF that contains 1st column Genome ID and remaining columns each of the group to be visualized. Shapes and colors are set manually in iTOL

# Old code
df_clinker_alignment_manual = pd.read_csv(dir_clinker + 'alignments_manual.csv')
df_clinker_alignment_manual = df_clinker_alignment_manual.iloc[:,1:6].transpose().reset_index().drop('index', axis=1).transpose()
df_clinker_alignment_manual['id'] = np.arange(len(df_clinker_alignment_manual)) // 1+ 1
df_clinker_alignment_manual = df_clinker_alignment_manual.set_index('id').transpose()


# create clean gene dictionary
df_clinker_alignment_manual = df_clinker_alignment_manual.transpose().to_dict('tight')
gene_groups = df_clinker_alignment_manual['data']
clean_gene_group_list = []
for group in gene_groups:
    group = [x for x in group if str(x) != 'nan']
    clean_gene_group_list.append(group)


# Code written together with thom to create the gene matrix automatically
df_genes_list = pd.read_csv(dir_clinker +'alignments.csv',header=None,usecols=[0,1]).dropna()
df_genes_list.rename( columns={0 :'Pair_gene_1'}, inplace=True )
df_genes_list.rename( columns={1 :'Pair_gene_2'}, inplace=True )
# Create an empty list
genes_list =[]
# Iterate over each row
for rows in df_genes_list.itertuples():
    # Create list for the current row
    each_pair =[rows.Pair_gene_1, rows.Pair_gene_2]
    # append the list to the final list
    genes_list.append(each_pair)

gene_group_list = [{genes_list[0][0]}]
for genes in genes_list:
    genes_added = False
    for gene_group in gene_group_list:
        if genes[0] in gene_group or genes[1] in gene_group:
            gene_group.add(genes[1])
            gene_group.add(genes[0])
            genes_added = True
    if genes_added == False:
        gene_group_list.extend([{genes[0]}])
        for gene_group in gene_group_list:
            if genes[0] in gene_group or genes[1] in gene_group:
                gene_group.add(genes[1])
                gene_group.add(genes[0])
sorted(gene_group_list)
gene_group_list.sort(key = len, reverse=True)

clean_gene_group_dictionary = {}
for index, group in enumerate(gene_group_list, start = 1):
    x = str(index).zfill(3) + "-" + str(len(group)).zfill(2)
    temp_group_dict = dict.fromkeys(group, x)
    clean_gene_group_dictionary.update(temp_group_dict)

In [91]:


# loading in cBLASTER results into a list of gene hits to combine with clinker output
df2_cblaster = df_cblaster.set_index('Organism')
df2_cblaster = df2_cblaster.iloc[:,10:]
    # DATA LOSS!!! REMOVAL OF DUBLICATES!!! Organism cant be used as index incase same cluster repeats 2x or more times in organism.
df2_cblaster = df2_cblaster[~df2_cblaster.index.duplicated(keep='first')]
df2_cblaster['Organism'] = df2_cblaster.index
df2_cblaster
df2_cblaster = pd.melt(df2_cblaster, id_vars=['Organism'],
        var_name='Annotation', value_name='%_Identity')
df2_cblaster = df2_cblaster.dropna(how='any')
df2_cblaster = df2_cblaster[(df2_cblaster != 0).all(1)]


df2_cblaster['gene group'] = df2_cblaster["Annotation"].map(clean_gene_group_dictionary)
df2_cblaster = df2_cblaster.dropna(how='any').drop(['Annotation'], axis=1)
# error caused by one gene having two homologue copies and this appearing twice in the dictionary
# keep only first indexed gene incase there are two genes in the organism belonging to the same cluster.
df2_cblaster = df2_cblaster.pivot_table(index ='Organism', columns='gene group', values='%_Identity').fillna(0)
df2_cblaster.to_csv(dir_iTOL+"shape_plot.csv")

hex_df = pd.DataFrame(list(df2_cblaster.columns), columns=['Field labels'])
hex_df[['Gene group', 'number of genes']] = hex_df['Field labels'].str.split("-", expand = True)
hex_df['number of genes'] = hex_df['number of genes'].astype('int')
hex_codes = []
for gene_number in hex_df['number of genes']:
    color_strength = gene_number/max(hex_df['number of genes'])
    R_color = 255 - int(255*color_strength)
    G_color = 171
    B_color = 210
    color_code = '#' + "".join(hex(val).split("0x")[1].zfill(2) for val in [R_color, G_color, B_color])
    hex_codes.append(color_code)

hex_df['Field colors'] = hex_codes
hex_df=hex_df.drop(columns=['Gene group', 'number of genes']).transpose()

hex_df.to_csv(dir_iTOL+"headder.csv")



In [79]:
df = pd.DataFrame([['Dollar Sales', 'Unit Sales', 'Dollar Sales', 'Unit Sales'], 
                   [1, 2, 3, 4], [5, 6, 7, 8]], columns=[2016, 2016, 2015, 2015])

new_labels = pd.MultiIndex.from_arrays([df.columns, df.iloc[0]], names=['Year', 'Sales'])
df1 = df.set_axis(new_labels, axis=1).iloc[1:]

